In [ ]:
from project_version_2 import *
plt.imshow(stored[0])

#Information about the first generated image: 
print(list_torch[0])
print(type(list_torch[0]))
print(list_torch[0].size())
print(list_torch[0].ndim)   
       